In [1]:
!pip -q install -Uqq datasets pythainlp==2.2.4 transformers==4.4.0 tensorflow==2.4.0 tensorflow_text emoji seqeval sentencepiece fuzzywuzzy
!npx degit --force https://github.com/vistec-AI/thai2transformers#dev

     |████████████████████████████████| 365 kB 9.7 MB/s 
     |████████████████████████████████| 8.9 MB 26.8 MB/s 
     |████████████████████████████████| 2.1 MB 58.5 MB/s 
     |████████████████████████████████| 394.7 MB 16 kB/s 
     |████████████████████████████████| 4.6 MB 57.9 MB/s 
     |████████████████████████████████| 197 kB 89.7 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 67.0 MB/s 
     |████████████████████████████████| 965 kB 70.9 MB/s 
     |████████████████████████████████| 880 kB 82.0 MB/s 
     |████████████████████████████████| 3.3 MB 53.7 MB/s 
     |████████████████████████████████| 3.8 MB 62.3 MB/s 
     |████████████████████████████████| 462 kB 90.9 MB/s 
     |████████████████████████████████| 14.8 MB 65.6 MB/s 
     |████████████████████████████████| 132 kB 99.5 MB/s 
     |████████████████████████████████| 2.9 MB 66.0 MB/s 
     |████████████████████████████████| 115 kB 73.3 MB/s 
     |██████████

In [1]:
%load_ext autoreload
%autoreload 2

import pythainlp, transformers
pythainlp.__version__, transformers.__version__ #fix pythainlp to stabilize word tokenization for metrics

('2.2.4', '4.4.0')

In [2]:
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)

#transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

#thai2transformers
import thai2transformers
from thai2transformers.metrics import (
    squad_newmm_metric,
    question_answering_metrics,
)
from thai2transformers.preprocess import (
    prepare_qa_train_features
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

In [3]:
model_names = [
    'wangchanberta-base-att-spm-uncased',
    'xlm-roberta-base',
    'bert-base-multilingual-cased',
    'wangchanberta-base-wiki-newmm',
    'wangchanberta-base-wiki-ssg',
    'wangchanberta-base-wiki-sefr',
    'wangchanberta-base-wiki-spm',
]

tokenizers = {
    'wangchanberta-base-att-spm-uncased': AutoTokenizer,
    'xlm-roberta-base': AutoTokenizer,
    'bert-base-multilingual-cased': AutoTokenizer,
    'wangchanberta-base-wiki-newmm': ThaiWordsNewmmTokenizer,
    'wangchanberta-base-wiki-ssg': ThaiWordsSyllableTokenizer,
    'wangchanberta-base-wiki-sefr': FakeSefrCutTokenizer,
    'wangchanberta-base-wiki-spm': ThaiRobertaTokenizer,
    
}
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
#@title Choose Pretrained Model
model_name = "xlm-roberta-base" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm"]

#create tokenizer
tokenizer = tokenizers[model_name].from_pretrained(
                f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

In [4]:
#parameterizing columns
CONTEXT_COL = 'context'
QUESTION_COL = 'question'
ANSWERS_COL = 'answers'
TEXT_COL = 'answer'
START_COL = 'answer_begin_position'
END_COL = 'answer_end_position'
QUESTION_ID_COL = 'question_id'

In [5]:
from datasets import load_dataset
thaiqa_ = load_dataset('thaiqa_squad')

Reusing dataset thaiqa_squad (/root/.cache/huggingface/datasets/thaiqa_squad/thaiqa_squad/1.0.0/fce14864b511d48464540780f328f4b415746b63f2fd934ad0b06c3eead7787b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from lxml import etree
parser = etree.XMLParser(recover=True)
def extract_text(body):
  tree = etree.fromstring(body, parser=parser)
  return tree.text

In [7]:
map_object = map(extract_text, thaiqa_['train']['context'])
new_context = list(map_object)
new_context[0]

"เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกำกับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทำที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคำ สาขาเพลงประกอบยอดเยี่ยม จากเพลง Benji's Theme (I Feel Love) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, Benji (ภาพยนตร์) - พ.ศ. 2520, For the Love of Benji (ภาพยนตร์) - พ.ศ. 2521, Benji's Very Own Christmas Story (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, Oh Heavenly Dog (ภาพยนตร์) - พ.ศ. 2523, Benji at Work (ภาพยนตร์โทรทัศน์) - พ.ศ. 2524, Benji Takes a Dive at Marineland (ภาพยนตร์โทรทัศน์) - พ.ศ. 2526, Benji, Zax  the Alien Prince (ภาพยนตร์ซีรีส์) - พ.ศ. 2530, Benji the Hunted (ภาพยนตร์) - พ.ศ. 254

In [8]:
new_question = list(thaiqa_['train']['question'])
new_question[0]

'สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร'

In [9]:
for i in range(len(new_question)):
  new_question[i] = new_question[i].replace(' ','')
new_question[0]

'สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่องBenjiที่ออกฉายในปีพ.ศ.2517มีชื่อว่าอะไร'

In [10]:
for i in range(len(new_context)):
  new_context[i] = new_context[i].replace(' ','')
new_context[0]

"เบนจี้เบนจี้()เป็นชื่อตัวละครหมาพันทางแสนรู้ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกำกับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทำที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์และได้รางวัลลูกโลกทองคำสาขาเพลงประกอบยอดเยี่ยมจากเพลงBenji'sTheme(IFeelLove)ร้องโดยชาร์ลีริชหมาที่แสดงเป็นเบนจี้ตัวแรกชื่อว่าฮิกกิ้นส์(พ.ศ.2502-พ.ศ.2518)มีอายุถึง15ปีแล้วในขณะแสดงหลังจากภาพยนตร์ออกฉายได้ไม่นานมันก็ตายในปีพ.ศ.2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์.-พ.ศ.2517,Benji(ภาพยนตร์)-พ.ศ.2520,FortheLoveofBenji(ภาพยนตร์)-พ.ศ.2521,Benji'sVeryOwnChristmasStory(ภาพยนตร์โทรทัศน์)-พ.ศ.2523,OhHeavenlyDog(ภาพยนตร์)-พ.ศ.2523,BenjiatWork(ภาพยนตร์โทรทัศน์)-พ.ศ.2524,BenjiTakesaDiveatMarineland(ภาพยนตร์โทรทัศน์)-พ.ศ.2526,Benji,ZaxtheAlienPrince(ภาพยนตร์ซีรีส์)-พ.ศ.2530,BenjitheHunted(ภาพยนตร์)-พ.ศ.2547,Benji:OfftheLeash!(ภาพยนตร์)-พ.ศ.2550,Benji:TheBarkening(ภาพยนตร์)"

In [11]:
thaiqa_['train']['context'][0]

'<doc id="115035" url="https://th.wikipedia.org/wiki?curid=115035" title="เบนจี้">เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกำกับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทำที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคำ สาขาเพลงประกอบยอดเยี่ยม จากเพลง Benji\'s Theme (I Feel Love) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, Benji (ภาพยนตร์) - พ.ศ. 2520, For the Love of Benji (ภาพยนตร์) - พ.ศ. 2521, Benji\'s Very Own Christmas Story (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, Oh Heavenly Dog (ภาพยนตร์) - พ.ศ. 2523, Benji at Work (ภาพยนตร์โทรทัศน์) - พ.ศ. 2524, Benji Takes a Dive at Marineland (ภาพยนตร์โทรทัศน์) - พ.ศ. 2526, Benji, Zax & t

In [12]:
new_context[0]

"เบนจี้เบนจี้()เป็นชื่อตัวละครหมาพันทางแสนรู้ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกำกับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทำที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์และได้รางวัลลูกโลกทองคำสาขาเพลงประกอบยอดเยี่ยมจากเพลงBenji'sTheme(IFeelLove)ร้องโดยชาร์ลีริชหมาที่แสดงเป็นเบนจี้ตัวแรกชื่อว่าฮิกกิ้นส์(พ.ศ.2502-พ.ศ.2518)มีอายุถึง15ปีแล้วในขณะแสดงหลังจากภาพยนตร์ออกฉายได้ไม่นานมันก็ตายในปีพ.ศ.2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์.-พ.ศ.2517,Benji(ภาพยนตร์)-พ.ศ.2520,FortheLoveofBenji(ภาพยนตร์)-พ.ศ.2521,Benji'sVeryOwnChristmasStory(ภาพยนตร์โทรทัศน์)-พ.ศ.2523,OhHeavenlyDog(ภาพยนตร์)-พ.ศ.2523,BenjiatWork(ภาพยนตร์โทรทัศน์)-พ.ศ.2524,BenjiTakesaDiveatMarineland(ภาพยนตร์โทรทัศน์)-พ.ศ.2526,Benji,ZaxtheAlienPrince(ภาพยนตร์ซีรีส์)-พ.ศ.2530,BenjitheHunted(ภาพยนตร์)-พ.ศ.2547,Benji:OfftheLeash!(ภาพยนตร์)-พ.ศ.2550,Benji:TheBarkening(ภาพยนตร์)"

In [ ]:
answer_start = []
for i in range(len(thaiqa_['train']['answers'])):
  answer_start.append(new_context[i].find(thaiqa_['train']['answers'][i]['answer'][0]))

answer_start

In [14]:
answer_start[0]

412

In [15]:
thaiqa_['train']['answers'][0]['answer']

['ฮิกกิ้นส์']

In [16]:
new_context[0][412:]

"ฮิกกิ้นส์(พ.ศ.2502-พ.ศ.2518)มีอายุถึง15ปีแล้วในขณะแสดงหลังจากภาพยนตร์ออกฉายได้ไม่นานมันก็ตายในปีพ.ศ.2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์.-พ.ศ.2517,Benji(ภาพยนตร์)-พ.ศ.2520,FortheLoveofBenji(ภาพยนตร์)-พ.ศ.2521,Benji'sVeryOwnChristmasStory(ภาพยนตร์โทรทัศน์)-พ.ศ.2523,OhHeavenlyDog(ภาพยนตร์)-พ.ศ.2523,BenjiatWork(ภาพยนตร์โทรทัศน์)-พ.ศ.2524,BenjiTakesaDiveatMarineland(ภาพยนตร์โทรทัศน์)-พ.ศ.2526,Benji,ZaxtheAlienPrince(ภาพยนตร์ซีรีส์)-พ.ศ.2530,BenjitheHunted(ภาพยนตร์)-พ.ศ.2547,Benji:OfftheLeash!(ภาพยนตร์)-พ.ศ.2550,Benji:TheBarkening(ภาพยนตร์)"

In [17]:
new_context[0][412:]

"ฮิกกิ้นส์(พ.ศ.2502-พ.ศ.2518)มีอายุถึง15ปีแล้วในขณะแสดงหลังจากภาพยนตร์ออกฉายได้ไม่นานมันก็ตายในปีพ.ศ.2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์.-พ.ศ.2517,Benji(ภาพยนตร์)-พ.ศ.2520,FortheLoveofBenji(ภาพยนตร์)-พ.ศ.2521,Benji'sVeryOwnChristmasStory(ภาพยนตร์โทรทัศน์)-พ.ศ.2523,OhHeavenlyDog(ภาพยนตร์)-พ.ศ.2523,BenjiatWork(ภาพยนตร์โทรทัศน์)-พ.ศ.2524,BenjiTakesaDiveatMarineland(ภาพยนตร์โทรทัศน์)-พ.ศ.2526,Benji,ZaxtheAlienPrince(ภาพยนตร์ซีรีส์)-พ.ศ.2530,BenjitheHunted(ภาพยนตร์)-พ.ศ.2547,Benji:OfftheLeash!(ภาพยนตร์)-พ.ศ.2550,Benji:TheBarkening(ภาพยนตร์)"

In [19]:
context = new_context[0]
question = thaiqa_["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

"<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s> เบนจี้เบนจี้()เป็นชื่อตัวละครหมาพันทางแสนรู้ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกํากับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์และได้รางวัลลูกโลกทองคําสาขาเพลงประกอบยอดเยี่ยมจากเพลงBenji'sTheme(IFeelLove)ร้องโดยชาร์ลีริชหมาที่แสดงเป็นเบนจี้ตัวแรกชื่อว่าฮิกกิ้นส์(พ.ศ.2502-พ.ศ.2518)มีอายุถึง15ปีแล้วในขณะแสดงหลังจากภาพยนตร์ออกฉายได้ไม่นานมันก็ตายในปีพ.ศ.2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์.-พ.ศ.2517,Benji(ภาพยนตร์)-พ.ศ.2520,FortheLoveofBenji(ภาพยนตร์)-พ.ศ.2521,Benji'sVeryOwnChristmasStory(ภาพยนตร์โทรทัศน์)-พ.ศ.2523,OhHeavenlyDog(ภาพยนตร์)-พ.ศ.2523,BenjiatWork(ภาพยนตร์โทรทัศน์)-พ.ศ.2524,BenjiTakesaDiveatMarineland(ภาพยนตร์โทรทัศน์)-พ.ศ.2526,Benji,ZaxtheAlienPrince(ภาพยนตร์ซีรีส์)-พ.ศ.2530,BenjitheHunted(ภาพยนตร์)-พ.ศ.2547,Benji:Offt

In [20]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)

for ids in inputs["input_ids"]:
    print((tokenizer.decode(ids))) # จะเห็นว่าประโยคที่สองคือการตัดมาจากอันแรก

<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s> เบนจี้เบนจี้()เป็นชื่อตัวละครหมาพันทางแสนรู้ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกํากับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทก</s>
<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s>ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกํากับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัล</s>
<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s>รรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์และได้รางวัลลูกโลกทองคําสาขาเพลงประกอบยอดเยี่ยมจากเพลงBenji'</s>
<s

In [39]:
len(inputs['input_ids'])

18

In [40]:
inputs['overflow_to_sample_mapping'] 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [41]:
len(inputs['overflow_to_sample_mapping'])

18

In [ ]:
answers = thaiqa_["train"][0:2]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer_start[0]
    end_char = answer_start[0] + len(answer["answer"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions
for ids in inputs["input_ids"]:
    print((tokenizer.decode(ids))) # จะเห็นว่าประโยคที่สองคือการตัดมาจากอันแรก
print(start_positions)
print(end_positions)

<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s> เบนจี้เบนจี้()เป็นชื่อตัวละครหมาพันทางแสนรู้ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกํากับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทก</s>
<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s>ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบทและกํากับโดยโจแคมป์ในช่วงทศวรรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัล</s>
<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s>รรษ1970ถึง1980ภาพยนตร์เรื่องแรกในชุดใช้ชื่อเรื่องว่าเบนจี้เช่นเดียวกับตัวละครถ่ายทําที่เมืองดัลลัสรัฐเทกซัสฉายครั้งแรกในปีพ.ศ.2517ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์และได้รางวัลลูกโลกทองคําสาขาเพลงประกอบยอดเยี่ยมจากเพลงBenji'</s>
<s

In [ ]:
max_length = 400
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        new_question,
        new_context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = thaiqa_['train']["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer_start[0]
        end_char = answer_start[0] + len(answer["answer"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = thaiqa_["train"].map( # จัดการ training data
    preprocess_training_examples,
    batched=True,
    remove_columns=thaiqa_["train"].column_names,
)
len(thaiqa_["train"]), len(train_dataset) # ทำการตัดคำทั้งหมดพร้อมทั้งแบ่งเป็นก้อนๆ

  0%|          | 0/4 [00:00<?, ?ba/s]

(4000, 79572)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 79572
})

In [ ]:
thaiqa_['validation']['question_id']

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        new_question,
        new_context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(thaiqa_['validation']['question_id'][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = thaiqa_["validation"].map( # จัดการ validation data
    preprocess_training_examples,
    batched=True,
    remove_columns=thaiqa_["validation"].column_names,
)
len(thaiqa_["validation"]), len(validation_dataset) # ทำการตัดคำทั้งหมดพร้อมทั้งแบ่งเป็นก้อนๆ

  0%|          | 0/1 [00:00<?, ?ba/s]

(74, 14595)

In [ ]:
validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 14595
})

In [ ]:
small_eval_set = thaiqa_["validation"].select(range(74)) # ดาต้าขนาดเล็กก็พอแล้ว
trained_checkpoint = "airesearch/wangchanberta-base-wiki-20210520-spm-finetune-qa" # โมเดลที่เทรนมาเรียบร้อยแล้ว

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint) # โหลด Tokenizer ของตัว finetune มาโดยใช้คำสั่ง?
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=thaiqa_["validation"].column_names,
)

In [ ]:
tokenizer = tokenizers[model_name].from_pretrained(
                f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,) # เอาตัวตัดคำอันเดิมกลับมา

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch) # ได้คู่ของ start,end ที่มาจากโมเดลที่เทรนมาเรียบร้อยแล้ว!

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy() # เอาออกจาก gpu เพราะว่าแค่เอาเลขมาเทียบ ไม่จำเป็นต้องลง gpu

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["question_id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
metric = squad_newmm_metric # โหลด F1 กับ exact match มา

In [ ]:
theoretical_answers = [ 
    {"id": ex["question_id"], "answers": {'text': ex[ANSWERS_COL][TEXT_COL],
                           'answer_start':ex[ANSWERS_COL][START_COL]}} for ex in small_eval_set
]

In [ ]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["question_id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["question_id"], "answers": {'text': ex[ANSWERS_COL][TEXT_COL],
                           'answer_start':ex[ANSWERS_COL][START_COL]}} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(
            f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
            revision='main',)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

In [ ]:
batch_size = 32
learning_rate = 2e-5

args = TrainingArguments(
    f"finetune_iapp_thaiqa",
    evaluation_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    num_train_epochs=2,
    warmup_ratio=0.2,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=3,
    load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss
500,2.414200
1000,0.679100
1500,0.581600
2000,0.536900
2500,0.492100
3000,0.447100
3500,0.425400
4000,0.396900
4500,0.346800


TrainOutput(global_step=4974, training_loss=0.6674433095799328, metrics={'train_runtime': 6619.4659, 'train_samples_per_second': 0.751, 'total_flos': 1.059725613780864e+17, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 388463, 'init_mem_gpu_alloc_delta': 1109893120, 'init_mem_cpu_peaked_delta': 19163, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1372599, 'train_mem_gpu_alloc_delta': 3348135936, 'train_mem_cpu_peaked_delta': 4202752, 'train_mem_gpu_peaked_delta': 11101483520})